In [5]:
from keras.models import Sequential
from keras.layers import Conv2D,ZeroPadding2D,Activation,Input,concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D,AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda,Flatten,Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

K.set_image_data_format('channels_first')

import time
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
import fr_utils
from inception_blocks_v2 import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

np.set_printoptions(threshold=np.nan)

In [7]:
FRmodel = faceRecoModel(input_shape=(3,96,96))
print('参数数量：'+str(FRmodel.count_params()))

参数数量：3743280


In [15]:
def triplet_loss(y_true,y_pred,alpha=0.2):
    '''
    三元组损失函数
    参数：
        y_true-- true标签，当你在Keras里定义了一个损失函数的时候需要他
        y_pred--列表类型，包含以下参数
            anchor--给定的“anchor”图像的编码，维度为(None,128)
            positive--给定的“positive”图像的编码，维度为(None,128)
            negative--给定的“negative”图像的编码，维度为(None,128)
        alpha--超参数，阈值
        
    返回:
        loss --实数，损失值
    '''
    # 获取图像编码
    anchor,positive,negative = y_pred[0],y_pred[1],y_pred[2]
    # 计算anchor和positive的编码距离
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,positive)),axis=-1)
    # 计算anchor和negative的编码距离
    neg_dis = tf.reduce_sum(tf.square(tf.subtract(anchor,negative)),axis=-1)
    # 减去之前的两个距离，加上alpha
    basic_loss = tf.add(tf.subtract(pos_dist,neg_dis),alpha)
    
    loss = tf.reduce_sum(tf.maximum(basic_loss,0))
    
    return loss

In [16]:
with tf.Session() as test:
    tf.set_random_seed(1)
    y_true = (None,None,None)
    y_pred = (tf.random_normal([3,128],mean=6,stddev=0.1,seed=1),
             tf.random_normal([3,128],mean=1,stddev=1,seed=1),
             tf.random_normal([3,128],mean=3,stddev=4,seed=1),)
    loss = triplet_loss(y_true,y_pred)
    print(str(loss.eval()))

528.1432


In [17]:
start_time = time.clock()

FRmodel.compile(optimizer='adam',loss=triplet_loss,metrics=['accuracy'])
fr_utils.load_weights_from_FaceNet(FRmodel)

end_time = time.clock()
minum = end_time - start_time
print('执行用时：' + str(minum/60)+'分钟，'+str(minum%60)+'秒。')

执行用时：2.5898244409852分钟，35.389466459112015秒。


In [18]:
database = {}
database["danielle"] = fr_utils.img_to_encoding("images/danielle.png", FRmodel)
database["younes"] = fr_utils.img_to_encoding("images/younes.jpg", FRmodel)
database["tian"] = fr_utils.img_to_encoding("images/tian.jpg", FRmodel)
database["andrew"] = fr_utils.img_to_encoding("images/andrew.jpg", FRmodel)
database["kian"] = fr_utils.img_to_encoding("images/kian.jpg", FRmodel)
database["dan"] = fr_utils.img_to_encoding("images/dan.jpg", FRmodel)
database["sebastiano"] = fr_utils.img_to_encoding("images/sebastiano.jpg", FRmodel)
database["bertrand"] = fr_utils.img_to_encoding("images/bertrand.jpg", FRmodel)
database["kevin"] = fr_utils.img_to_encoding("images/kevin.jpg", FRmodel)
database["felix"] = fr_utils.img_to_encoding("images/felix.jpg", FRmodel)
database["benoit"] = fr_utils.img_to_encoding("images/benoit.jpg", FRmodel)
database["arnaud"] = fr_utils.img_to_encoding("images/arnaud.jpg", FRmodel)

In [23]:
def verify(image_path,identify,database,model):
    '''
    对identify和image_path的编码进行验证
    参数：
        image_path--摄像头的图片
        identify--想要验证的人的名字
        database--字典类型，包含了成员信息和对应的编码
        model--在keras的模型的实例
    返回：
        dist--摄像头图片和数据库图片编码的差距
        is_open_door--boolean,是否该开门
    '''
    # 计算图像的编码，使用fr_utils.img_to_encoding(img_path,model)
    encoding = fr_utils.img_to_encoding(image_path,model)
    # 计算与数据库中保存编码的差距
    dist = np.linalg.norm(encoding - database[identify])
    if dist < 0.7:
        print('欢迎,'+str(identify)+'回家')
        is_door_open = True
    else:
        print('经过验证，你与'+str(identify)+'不符！')
        is_door_open = False
        
    return dist,is_door_open

In [24]:
verify('images/camera_0.jpg','younes',database,FRmodel)

欢迎,younes回家


(0.67100745, True)

In [25]:
verify('images/camera_2.jpg','kian',database,FRmodel)

经过验证，你与kian不符！


(0.85800153, False)

In [29]:
def who_is_it(img_path,database,model):
    '''
    根据指定的图片来进行人脸识别
    参数：
        img_path--图像地址
        database--字典类型，包含了成员信息和对应的编码
        model--在keras的模型的实例
    返回：
        min_dist -- 在数据库中与指定图像最相近的编码。
        identity -- 字符串类型，与min_dist编码相对应的名字。
    '''
    encoding = fr_utils.img_to_encoding(img_path,model)
    min_dist = 100
    
    for (name,db_enc) in database.items():
        dist = np.linalg.norm(encoding-db_enc)
        if dist < min_dist:
            min_dist = dist
            identify = name
    
    if min_dist > 0.7:
        print('抱歉，你不在数据库的信息中')
    else:
        print('姓名'+str(identify)+'差距：'+str(min_dist))
    return min_dist,identify

In [31]:
who_is_it('images/camera_0.jpg',database,FRmodel)

姓名younes差距：0.67100745


(0.67100745, 'younes')